In [1]:
import cv2
import numpy as np
from keras.models import model_from_json



In [2]:

emotion_dict = {0: "Happy", 1: "Neutral", 2: "Sad"}

# load json and create model
json_file = open('emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

# load weights into new model
emotion_model.load_weights("emotion_model.h5")
print("Loaded model from disk")


Loaded model from disk


In [3]:
def detect_face(img):
    face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')   
    face_img = img.copy()
    face_rects = face_cascade.detectMultiScale(face_img,scaleFactor=1.3, minNeighbors=5)
    for (x,y,w,h) in face_rects:
        cv2.rectangle(face_img,(x,y),(x+w,y+h),(0,255,0),5)

    return (x,y,w,h)

In [4]:
# tracker = cv2.legacy.TrackerKCF_create()
tracker = cv2.TrackerCSRT_create()
tracker_name = str(tracker).split()[0][8:]

# happy = 0.9
# sad = 0.8
# neutral = 1.0

happy_count = 0
sad_count = 0
neutral_count = 0
face_front = 0
frame_count = 0

cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture("video2.mp4")

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         continue
#     frame = cv2.resize(frame, (1080, 720))
#     try:
#         roi = detect_face(frame)
#         break
#     except Exception as e:
#         print("Couldn't detect face, trying again!")
#         print(e)
        # cap.release()
ret, frame = cap.read()
frame = cv2.resize(frame, (1080, 720))
roi = detect_face(frame)
ret = tracker.init(frame, roi)


UnboundLocalError: local variable 'x' referenced before assignment

In [ ]:
try:
    while True:
        frame_count += 1

        ret, frame = cap.read()
        frame = cv2.resize(frame, (1080, 720))
        # frame = cv2.flip(frame, 1)

        success, roi = tracker.update(frame)

        (x,y,w,h) = tuple(map(int,roi))
        
        if not ret:
            break
        # face_detector = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # # detect faces available on camera
        # num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

        # # take each face available on the camera and Preprocess it
        # for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), 0)

        if len(cropped_img) > 0:
            face_front += 1

        # predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img, verbose=0)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        if emotion_dict[maxindex] == 'Happy':
            happy_count += 1
        elif emotion_dict[maxindex] == 'Sad':
            sad_count += 1
        elif emotion_dict[maxindex] == 'Neutral':
            neutral_count += 1

            
        # Draw Rectangle as Tracker moves
        if success:
            # Tracking success
            p1 = (x, y)
            p2 = (x+w, y+h)
            cv2.rectangle(frame, p1, p2, (0,0,255), 3)
        else :
            # Tracking failure
            cv2.putText(frame, "Failure to Detect Tracking!!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)

        # Display tracker type on frame
        cv2.putText(frame, tracker_name, (20,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),3)    

        cv2.imshow(tracker_name, frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except Exception as e:
    print(e)
    
cap.release()
cv2.destroyAllWindows()

print(frame_count)
print(face_front)
print("Happy = ", happy_count)
print("Sad = ", sad_count)
print("Neutral = ", neutral_count)

avg_emotion = (happy_count * 0.9 + sad_count * 0.8 + neutral_count) / frame_count
print("Average Emotion Index = ", avg_emotion)
front_view = face_front/frame_count
print("Front View Rate = ", front_view)

attention = (avg_emotion * front_view) * 100

print("Attention Rate = ", attention)


OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

38
37
Happy =  4
Sad =  7
Neutral =  26
Average Emotion Index =  0.9263157894736843
Front View Rate =  0.9736842105263158
Attention Rate =  90.19390581717452
